In [18]:
import ee
import geemap
import pandas as pd
from plotly import graph_objects as go
import plotly.express as px
import geojson

In [19]:
ee.Authenticate()
ee.Initialize()

In [20]:
flood_collection = ee.FeatureCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")

interest_properties = {
    "id": "flood_id",
    "dfo_country": "primary_country",
    "dfo_severity": "severity",
    "system:time_start": "start_date",
    "system:time_end": "end_date",
    "dfo_centroid_x": "center_lon",
    "dfo_centroid_y": "center_lat",
}


def extract_interest(feature):
    old_keys = ee.List(list(interest_properties.keys()))
    new_keys = ee.List(list(interest_properties.values()))
    props = feature.toDictionary(old_keys).rename(old_keys, new_keys, True)
    return ee.Feature(None, props)


flood_properties = flood_collection.map(extract_interest)

geemap.ee_to_csv(flood_properties, filename="flood.csv")

In [21]:
df_flood = pd.read_csv("flood.csv")
df_flood.head()

,center_lat,center_lon,end_date,flood_id,primary_country,severity,start_date
0,-31.268059,143.697800,951868800000,1586,Australia,2.0,950832000000
1,-15.782624,47.295670,952732800000,1587,Madagascar,1.0,950745600000
2,46.763746,22.415404,956620800000,1595,Romania,2.0,954892800000
3,11.242567,105.063841,965865600000,1614,Thailand,1.0,963273600000
4,43.773883,132.057679,968544000000,1627,China,1.0,967593600000


In [22]:
df_flood.isnull().sum()

center_lat         0
center_lon         0
end_date           0
flood_id           0
primary_country    0
severity           0
start_date         0
dtype: int64

In [23]:
df_flood.describe()

,center_lat,center_lon,end_date,flood_id,severity,start_date
count,913.000000,913.000000,9.130000e+02,913.000000,913.000000,9.130000e+02
mean,15.395235,38.340958,1.224903e+12,3238.526835,1.297371,1.223187e+12
std,24.050427,78.687131,1.472525e+11,796.705451,0.392520,1.474857e+11
min,-45.953281,-156.215507,9.518688e+11,1586.000000,1.000000,9.507456e+11
25%,0.058024,-13.967824,1.103328e+12,2596.000000,1.000000,1.102291e+12
50%,18.877688,55.552555,1.197245e+12,3226.000000,1.000000,1.195085e+12
75%,32.886368,103.415798,1.330301e+12,3900.000000,1.500000,1.329696e+12
max,68.000870,178.075692,1.544400e+12,4711.000000,2.000000,1.543968e+12


In [24]:
with open("countries.geojson") as f:
    countries_boundary = geojson.load(f)

countries_ADM = [
    feature["properties"]["ADMIN"] for feature in countries_boundary["features"]
]

In [25]:
def check_country_exists_ADM(countries):
    return pd.DataFrame(
        {
            "primary_country": countries,
            "exists_in_ADM": countries.isin(countries_ADM),
        }
    )


df_temp = check_country_exists_ADM(pd.Series(df_flood["primary_country"].unique()))
df_temp[df_temp["exists_in_ADM"] == False]

,primary_country,exists_in_ADM
14,USA,False
18,Africa,False
23,Texas,False
24,Columbia,False
33,Burma,False
41,UK,False
66,Bosnia-Herzegovina,False
68,Venezulea,False
83,Serbia-Montenegro,False
92,Democratic Republic of Congo,False


In [26]:
country_to_adm = {
    "USA": "United States of America",
    "UK": "United Kingdom",
    "Burma": "Myanmar",
    "Tanzania": "United Republic of Tanzania",
    "Columbia": "Colombia",
    "Bosnia-Herzegovina": "Bosnia and Herzegovina",
    "Guatamala": "Guatemala",
    "Serbia": "Republic of Serbia",
    "Africa": "Central African Republic",  # Africa is a continent, not a country
    "Texas": "United States of America",  # Texas is a state in the USA
    "Venezulea": "Venezuela",
    "Serbia-Montenegro": "Republic of Serbia",  # No longer exists as a single country
    "Inda": "India",
    "Democratic Republic of Congo": "Democratic Republic of the Congo",
    "Tasmania": "Australia",  # Tasmania is a state in Australia
    "The Gambia": "Gambia",
    "Scotland": "United Kingdom",  # Scotland is part of the United Kingdom
    "Gaza": "Palestine",  # Gaza is a region in Palestine
    "Kazahkstan": "Kazakhstan",
    "Uruguay,": "Uruguay",
    "Bahamas": "The Bahamas"
}

def process_country_name(country):
    processed_country = country_to_adm.get(country)
    return processed_country if processed_country else country

df_flood["primary_country"] = df_flood["primary_country"].apply(process_country_name)
df_temp = check_country_exists_ADM(pd.Series(df_flood["primary_country"].unique()))
df_temp[df_temp["exists_in_ADM"] == False]

,primary_country,exists_in_ADM


In [27]:
df_flood_by_country = pd.DataFrame(
    df_flood["primary_country"].value_counts().reset_index(name="counts")
)
df_flood_by_country

,primary_country,counts
0,United States of America,87
1,India,85
2,China,52
3,Australia,47
4,Philippines,44
...,...,...
106,The Bahamas,1
107,Croatia,1
108,Burundi,1
109,Rwanda,1


In [28]:
hovertext = [
    f"Latitude: {lat}<br>Longitude: {lon}<br>Severity: {severity}"
    for lat, lon, severity in zip(df_flood['center_lat'], df_flood['center_lon'], df_flood['severity'])
]

scattergeo_trace = go.Scattergeo(
    lat=df_flood["center_lat"],
    lon=df_flood["center_lon"],
    mode="markers",
    marker=dict(
        size=12,
        opacity=0.8,
        autocolorscale=False,
        symbol="triangle-down",
        colorscale="Reds",
        cmin=0,
        color=df_flood["severity"],
        cmax=df_flood["severity"].max(),
        colorbar_title="Severity",
        colorbar_x = 1.15
    ),
    hovertext=hovertext
)

choropleth_trace = go.Choropleth(
    locations=df_flood_by_country["primary_country"],
    locationmode='country names',
    z=df_flood_by_country["counts"],
    colorscale="amp",
    colorbar_title="Occurence",
)


fig = go.Figure(data=[scattergeo_trace, choropleth_trace])

fig.update_geos(projection_type="orthographic")

fig.update_layout(margin={"r": 0, "t": 45, "l": 0, "b": 15})
fig.update_layout(
    title_text="Flood Severity and Occurence",
    title_x=0.5,
    geo=dict(
        showland=True,
        landcolor="rgb(95, 138, 92)", 
        showcountries=True,
        countrycolor="rgb(255, 255, 255)",  
        showocean=True,
        oceancolor="rgb(158,202,225)",  
    ),
)
fig.show()

fig = px.bar(df_flood_by_country, x="primary_country", y="counts", color="counts", title="Flood occurence by Country")
fig.update_layout(title_x = 0.5)
fig.show()

In [46]:
flood_collection = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")
landsat_collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterDate(
    "2000-02-17", "2018-12-10"
)

landsat_interest_bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B7"]
flood_interest_bands = [
    "flooded",
    "duration",
    "clear_views",
    "clear_perc",
    "jrc_perm_water",
]


def compose_landsat(image):
    duration = ee.Number.parse(
        image.getString("composite_type").replace("[^0-9.]+", "")
    )
    geometry = image.geometry()
    durationHalf = duration.divide(2).floor()
    start = ee.Date(image.get("system:time_start")).advance(
        durationHalf.multiply(-1), "day"
    )
    end = ee.Date(image.get("system:time_end")).advance(
        duration.subtract(durationHalf), "day"
    )
    landsat_image = (
        landsat_collection.filterDate(start, end)
        .select(ee.List(landsat_interest_bands))
        .filterBounds(geometry)
        .median()
        .clip(geometry)
    )
    return ee.Algorithms.If(
        landsat_image.bandNames().length(),
        landsat_image.addBands(image, ee.List(flood_interest_bands)),
        None,
    )
    

images = flood_collection.map(compose_landsat, True)
images.first()

In [40]:
vectorized_images = ee.List([])

def to_vector(image):
    vectorized_images.add(ee.FeatureCollection(
        image.toInt().reduceToVectors(
            geometry=image.geometry(),
            crs="EPSG:4326",
            scale=1000,
            geometryType="polygon",
            labelProperty="id",
            reducer=ee.Reducer.mean(),
        )
    ))
    return image
# geemap.ee_to_geotiff(images, output="output.tif", resolution=30)
# vectorized_images = images.map(to_vector)
vectorized_images.get(0).getInfo()

EEException: List.get: List is empty (index is 0).

In [44]:
geemap.ee_export_image(images.first(), "test.tif")

Generating URL ...
Please wait ...
Data downloaded to c:\Users\derri\PycharmProjects\SKIH3033_INFOVIS\test.tif


In [35]:
geemap.ee_export_vector(
    vectorized_images.first(),
    "test.csv",
)

ValueError: ee_object must be an ee.FeatureCollection

In [ ]:
from osgeo import gdal
import pandas as pd
dataset = gdal.Open('landsat.tif')
gdal.('output.xyz', dataset, format='XYZ')
df_p = pd.read_csv('output.xyz', sep=' ', header=None, names=['x', 'y', 'value'])
.to_csv('output.csv', index=False)